# Introduction

Ce premier notebook a pour but de construire une base de données combinant les données de NORMAN et de de CCSbase.  
- Les contaminants d'intérêt émergent de la base NORMAN-SusDat (https://www.norman-network.com/nds/susdat/)
- Les valeurs de CCS (Collision Cross Section) expérimentales et prédites (https://ccsbase.net/about)

# Import des packages

In [3]:
import pandas as pd
from rdkit import Chem

Pour assurer une correspondance précise entre les bases de données, nous devons standardiser la représentation des molécules :

In [2]:
def standardize_smiles(smiles):
    """
    Standardise les SMILES en utilisant RDKit
    Args:
        smiles (str): SMILES à standardiser
    Returns:
        str: SMILES standardisé ou None si invalide
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            return Chem.MolToSmiles(mol, isomericSmiles=True)
        else:
            return None
    except:
        return None

# Importation des bases de données sources et standardisation


In [12]:
norman = pd.read_csv("./norman.csv")
print(f"Nombre de composés dans la base Norman : {len(norman['Name'].unique())}")

Nombre de composés dans la base Norman : 120436


/tmp/ipykernel_212675/348901336.py:1: DtypeWarning: Columns (5,6,10,12,13,19,20,22,23,26,28,30,32,33,35,38,41,44,46,47,48,49,50,51,57,61,65,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  norman = pd.read_csv("./norman.csv")


In [20]:
# Chargement CCS expérimentale
ccs_exp = pd.read_excel("./ccs_base_experimental.xlsx")

# Filtrage TW et standardisation
ccs_exp_clean = ccs_exp[ccs_exp['CCS Type'] == 'TW'].copy()
ccs_exp_clean['SMILES'] = ccs_exp_clean['SMI'].apply(standardize_smiles)

# Sélection des colonnes pertinentes
ccs_exp_filtered = ccs_exp_clean[['SMILES', 'CCS', 'Adduct', 'M/Z']].copy()
ccs_exp_filtered.rename(columns={'CCS': 'ccs_exp', 'M/Z': 'mz_exp'}, inplace=True)

print(f"Nombre de composés CCS expérimentales après filtrage : {len(ccs_exp_filtered)}")

Nombre de composés CCS expérimentales après filtrage : 3318


In [ ]:
# Chargement CCS prédite
ccs_pred = pd.read_csv("CCSBase_Predicted database.csv")

# Standardisation des SMILES
ccs_pred_clean = ccs_pred.copy()
ccs_pred_clean['SMILES'] = ccs_pred_clean['smi'].apply(standardize_smiles)

# Sélection des colonnes pertinentes
ccs_pred_filtered = ccs_pred_clean[['SMILES', 'ccs', 'adduct', 'mz']].copy()
ccs_pred_filtered.rename(columns={'ccs': 'ccs_pred'}, inplace=True)

print(f"Nombre de composés CCS prédites après filtrage : {len(ccs_pred_filtered)}")

/tmp/ipykernel_212675/3642234017.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  ccs_pred = pd.read_csv("CCSBase_Predicted database.csv")


# Fusion des bases de données

In [ ]:
# Fusion avec CCS expérimentales
norman_with_exp = pd.merge(norman_clean, ccs_exp_filtered, on='SMILES', how='left')

# Fusion avec CCS prédites
norman_final = pd.merge(norman_with_exp, ccs_pred_filtered, on='SMILES', how='left')



# Sauvegarde 

In [ ]:
# Séparation des modes d'ionisation
norman_positive = norman_final[norman_final['adduct'].str.endswith('+')]
norman_negative = norman_final[norman_final['adduct'].str.endswith('-')]

# Affichage des statistiques par mode
print("Statistiques par mode d'ionisation:")
print(f"Nombre de composés en mode positif : {len(norman_positive)}")
print(f"Nombre de composés en mode négatif : {len(norman_negative)}")

# Sauvegarde au format HDF5
norman_positive.to_hdf("norman_all_ccs_pos_neg.h5", key="positive")
norman_negative.to_hdf("norman_all_ccs_pos_neg.h5", key="negative")
